# Doppler components

Many ways to specify the component of a spectral map.

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import starry

starry.config.lazy = False
starry.config.quiet = True

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import starry
from tqdm.auto import tqdm

## Define our data cube

In [ ]:
cube = np.ones((10, 10, 100))
wav0 = np.linspace(0, 1, 100)
cube[:, :] = 1 - np.exp(-0.5 * (wav0 - 0.25) ** 2 / 0.05 ** 2)
cube[4:6, 4:6] = 0.5 * (1 - np.exp(-0.5 * (wav0 - 0.75) ** 2 / 0.05 ** 2))

In [ ]:
cmap = plt.get_cmap("plasma")
fig, ax = plt.subplots(10, 10, sharex=True, sharey=True, figsize=(12, 6))
fig.subplots_adjust(hspace=0, wspace=0)
for i in range(10):
    for j in range(10):
        ax[i, j].set_facecolor(cmap(cube[i, j, 0]))
        ax[i, j].plot(cube[i, j], lw=1, color="k")
        ax[i, j].set_xticks([])
        ax[i, j].set_yticks([])
        for s in ["top", "right", "bottom", "left"]:
            ax[i, j].spines[s].set_alpha(0.1)
        ax[i, j].margins(0.1, 0.1)
for i in range(10):
    ax[i, 0].spines["left"].set_alpha(1)
    ax[i, -1].spines["right"].set_alpha(1)
for j in range(10):
    ax[0, j].spines["top"].set_alpha(1)
    ax[-1, j].spines["bottom"].set_alpha(1)

## SVD

In [ ]:
M = cube.reshape(100, 100)
U, s, VT = np.linalg.svd(M, full_matrices=False)
U = U[:, :2]
VT = VT[:2, :]
s = s[:2]
U = (U * s).reshape(10, 10, 2)

In [ ]:
print(np.max(np.abs(cube - U @ VT)))
fig, ax = plt.subplots(2, 2, figsize=(9, 6))
im = ax[0, 0].imshow(U[:, :, 0], vmin=U.min(), vmax=U.max(), aspect="auto")
plt.colorbar(im, ax=ax[0])
im = ax[0, 1].imshow(U[:, :, 1], vmin=U.min(), vmax=U.max(), aspect="auto")
plt.colorbar(im, ax=ax[1])
ax[1, 0].plot(VT[0])
ax[1, 0].plot(VT[1], alpha=0)
ax[1, 1].plot(VT[1])
ax[1, 1].plot(VT[0], alpha=0);

## Pin the images

In [ ]:
U = np.zeros((10, 10, 2))
U[:, :, 0] = 1
U[4:6, 4:6, 1] = 0.5
VT = np.zeros((2, 100))
VT[0] = 1 - np.exp(-0.5 * (wav0 - 0.25) ** 2 / 0.05 ** 2)
VT[1] = -2 * VT[0] + 1 - np.exp(-0.5 * (wav0 - 0.75) ** 2 / 0.05 ** 2)

In [ ]:
print(np.max(np.abs(cube - U @ VT)))
fig, ax = plt.subplots(2, 2, figsize=(9, 6))
im = ax[0, 0].imshow(U[:, :, 0], vmin=U.min(), vmax=U.max(), aspect="auto")
plt.colorbar(im, ax=ax[0])
im = ax[0, 1].imshow(U[:, :, 1], vmin=U.min(), vmax=U.max(), aspect="auto")
plt.colorbar(im, ax=ax[1])
ax[1, 0].plot(VT[0])
ax[1, 0].plot(VT[1], alpha=0)
ax[1, 1].plot(VT[1])
ax[1, 1].plot(VT[0], alpha=0);

## Pin the spectra

In [ ]:
VT = np.zeros((2, 100))
VT[0] = 1 - np.exp(-0.5 * (wav0 - 0.25) ** 2 / 0.05 ** 2)
VT[1] = 1 - np.exp(-0.5 * (wav0 - 0.75) ** 2 / 0.05 ** 2)
U = np.linalg.solve(VT @ VT.T, VT @ cube.reshape(100, 100).T).T.reshape(10, 10, 2)

In [ ]:
print(np.max(np.abs(cube - U @ VT)))
fig, ax = plt.subplots(2, 2, figsize=(9, 6))
im = ax[0, 0].imshow(U[:, :, 0], vmin=U.min(), vmax=U.max(), aspect="auto")
plt.colorbar(im, ax=ax[0])
im = ax[0, 1].imshow(U[:, :, 1], vmin=U.min(), vmax=U.max(), aspect="auto")
plt.colorbar(im, ax=ax[1])
ax[1, 0].plot(VT[0])
ax[1, 0].plot(VT[1], alpha=0)
ax[1, 1].plot(VT[1])
ax[1, 1].plot(VT[0], alpha=0);